In [1]:
import pandas as pd
import pandas as pd
from tqdm import tqdm
import pandas as pd
import pymongo
from googleapiclient.discovery import build
import pandas as pd
import seaborn as sns
from dotenv import load_dotenv, dotenv_values
import os
import json
import time
from langchain.document_loaders import YoutubeLoader
from langchain.llms import OpenAI
from langchain.chains.summarize import load_summarize_chain
import anthropic
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.llms.octoai_endpoint import OctoAIEndpoint
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from openai import OpenAI
import openai
import os
import together
from uuid import uuid4
from langchain_core.output_parsers import StrOutputParser
import ast
import itertools
import numpy as np
import logging
from typing import Any, Dict, List, Mapping, Optional
from langchain_community.embeddings import CohereEmbeddings
import cohere
from pydantic import Extra, Field, root_validator
from pymongo import MongoClient
import pinecone
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM
from langchain.llms.utils import enforce_stop_tokens
from langchain.utils import get_from_dict_or_env

from IPython.display import display, Markdown

from anthropic import Anthropic
import json
import re
import textwrap
from pprint import pprint
from langchain_anthropic import ChatAnthropic
from tqdm import tqdm
import tiktoken

df = pd.read_csv('coin_desk_details.csv')

/home/bilal326/Desktop/rag_project2/rag_project/my_virtual_env/lib/python3.10/site-packages/pinecone/data/index.py:1: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
df.head()

,https://www.coindesk.com/business/2023/11/13/asia-fund-picks-up-majority-stake-in-the-block-buys-out-shares-tied-to-sam-bankman-fried-loan/,Crypto News Site The Block Bought by Foresight Ventures in $70M Deal,2023-11-13 00:00:00,Unnamed: 3,"Most of the capital was used to buy out former CEO Mike McCaffrey's stake. McCaffrey resigned in December, 2022 after informing The Block's editorial team that the website had secretly been funded since 2020 by Sam Bankman-Fried’s Alameda Research. Bankman-Fried has since been found guilty of seven charges of fraud."
0,https://www.coindesk.com/business/2023/08/02/b...,Binance Nearly Shuttered U.S. Exchange to Prot...,2023-08-02 00:00:00,NaN,A person familiar with the matter told the out...
1,https://www.coindesk.com/consensus-magazine/20...,Alejandro Navia: Building the Future of Tokeni...,2023-04-25 00:00:00,NaN,It's very simple. To the outsider it looks lik...
2,https://www.coindesk.com/consensus-magazine/20...,Decentralized Media Via Web3: A Solution to Bi...,2023-03-24 00:00:00,NaN,It will be about connecting the offline world ...
3,https://www.coindesk.com/markets/2022/08/19/bi...,Bitcoin Retraces 3 Weeks of Gains as Analysts ...,2022-08-19 00:00:00,NaN,Two analysts with whom CoinDesk spoke said inv...
4,https://www.coindesk.com/markets/2022/08/10/ne...,NEAR Crypto Token Pumps After Coinbase Adds It...,2022-08-10 00:00:00,NaN,Near was created to operate faster and cheaper...


In [4]:
new_column_names = ['URL', 'Title', 'Publish Date', 'Authors', 'Text']
df.columns = new_column_names

In [14]:
df.sort_values(['Publish Date'], ascending=False)

,URL,Title,Publish Date,Authors,Text
0,https://www.coindesk.com/business/2023/08/02/b...,Binance Nearly Shuttered U.S. Exchange to Prot...,2023-08-02 00:00:00,NaN,A person familiar with the matter told the out...
1,https://www.coindesk.com/consensus-magazine/20...,Alejandro Navia: Building the Future of Tokeni...,2023-04-25 00:00:00,NaN,It's very simple. To the outsider it looks lik...
2,https://www.coindesk.com/consensus-magazine/20...,Decentralized Media Via Web3: A Solution to Bi...,2023-03-24 00:00:00,NaN,It will be about connecting the offline world ...
3,https://www.coindesk.com/markets/2022/08/19/bi...,Bitcoin Retraces 3 Weeks of Gains as Analysts ...,2022-08-19 00:00:00,NaN,Two analysts with whom CoinDesk spoke said inv...
4,https://www.coindesk.com/markets/2022/08/10/ne...,NEAR Crypto Token Pumps After Coinbase Adds It...,2022-08-10 00:00:00,NaN,Near was created to operate faster and cheaper...
...,...,...,...,...,...
13,https://www.coindesk.com/podcasts/markets-dail...,"Crypto News Roundup for Aug. 16, 2021",NaN,Adam B. Levine,Markets Daily Crypto Roundup\n\nCoinDesk Marke...
14,https://www.coindesk.com/podcasts/markets-dail...,"Crypto News Roundup for Aug. 12, 2021",NaN,Adam B. Levine,Markets Daily Crypto Roundup\n\nCoinDesk Marke...
15,https://www.coindesk.com/podcasts/markets-dail...,"Crypto News Roundup for Aug. 11, 2021",NaN,Adam B. Levine,Markets Daily Crypto Roundup\n\nCoinDesk Marke...
16,https://www.coindesk.com/podcasts/markets-dail...,"Crypto News Roundup for Aug. 6, 2021",NaN,Adam B. Levine,Markets Daily Crypto Roundup\n\nCoinDesk Marke...


In [6]:
df.shape

(693, 5)